### **README**

# What this does

This script allows you to download a merged results tile with no row limit, as a distinct `csv` file for each source query. You can then join the queries together per the merged query's merge rules elsewhere.

Ex) If you have a merged results title `foo` with 3 merged queries, this script would output 3 distinct csv files (one for each query) that ran with no row limit. 

# Notes
`DownloadUnlimitedMerged(dash_id, tile_title)` accepts 2 inputs:

- `dash_id`(string) - the ID of the dashboard
- `tile_title`(string) - the title of the merged results tile

Output files will be named `tile_titleq i`.

---

If using colab version, you will need to input your api url, client_id, and client_secret in the `## colab inputs ##` section. 

```
os.environ['LOOKERSDK_BASE_URL'] = 'https://example.looker.com'
os.environ['LOOKERSDK_API_VERSION'] = '3.1'
os.environ['LOOKERSDK_CLIENT_ID'] = 'client id here'
os.environ['LOOKERSDK_CLIENT_SECRET'] = 'client secret here'
```

If using python (local) version, these values should be set for the sdk per either an `.ini` file or as an environment variable. See Python SDK documentation for more information. https://pypi.org/project/looker-sdk/

In [4]:
### colab specific ###
# install the sdk
!pip install looker-sdk
# set environ variables
import os

## colab inputs ##
os.environ['LOOKERSDK_BASE_URL'] = ''
os.environ['LOOKERSDK_API_VERSION'] = '3.1'
os.environ['LOOKERSDK_CLIENT_ID'] = ''
os.environ['LOOKERSDK_CLIENT_SECRET'] = ''

In [2]:
# will need these for downloading the file from colab
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import files
from oauth2client.client import GoogleCredentials
# auth stuff for download
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [5]:
### import packages ###
# import and init
import looker_sdk
sdk = looker_sdk.init31("looker.ini")

import pandas as pd
import csv

In [44]:
## inputs ##

dash_id = ''
tile_title = ''


In [45]:
def DownloadUnlimitedMerged(dash_id, tile_title):

  data = sdk.search_dashboard_elements(dashboard_id=dash_id, title=tile_title )
  mergeid = data[0].merge_result_id

  # hit merge_query to get source query ids
  mergedata = sdk.merge_query(merge_query_id=mergeid)
  source_qs = mergedata.source_queries
  source_qids = []
  num_merged = len(source_qs)

  for x in range(num_merged):
    source_qids.append(source_qs[x].query_id)
    query_body = sdk.query(query_id=source_qids[x])
    elements_needed = ['model','view','fields','filters','sorts','limit','query_timezone']
    query_body['limit'] = '-1'
    listlen = len(elements_needed)

    final_body = {}

    for y in range(listlen):
      try:
          new_el = query_body[elements_needed[y]]
      except:
          i+=1
      else:
          final_body[elements_needed[y]] = new_el

    resultset = sdk.run_inline_query('csv', final_body)
    
    with open(tile_title + 'q '+str(x)+'.csv', 'w', newline='') as file:
              file.write(resultset)

    # colab stuff
    from google.colab import files 
    files.download(tile_title + 'q'+str(x)+'.csv')
    # end colab

DownloadUnlimitedMerged(dash_id, tile_title)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>